In [1]:
import numpy as np
from scipy.linalg import logm
from scipy.stats import multivariate_normal

In [2]:
import time

In [3]:
def kernel_covariance(x, l, s, noise=1e-6):
    """
    Covariance matrix with squared exponential kernel
    """
    t = x.shape[0]
    cov_sample = np.zeros((t, t))
    for i in range(t):
        for j in range(t):
            cov_sample[i, j] = s ** 2 * np.exp(-(x[i] - x[j]) ** 2 / (2 * l ** 2))
    cov_sample += np.eye(t) * noise  # add noise for numerical stability
    return cov_sample

In [4]:
def gp_marginal_likelihood(y, x, l, s):
    """
    Marginal likelihood of one Gaussian process (multivariate Normal)
    """
    t = y.shape[0]
    mu = np.repeat(0, t)
    cov = kernel_covariance(x, l, s)
    return multivariate_normal.pdf(y, mu, cov)

In [5]:
def calculate_p(l, s, Y, x, prior_params):
    """
    Calculate log prior and likelihood of n independent Gaussian processes (Y has shape [t, n])
    """
    #a, b, scale = prior_params
    #prior = l_gamma_prior(l, a, b)  # * s_half_cauchy_prior(s, scale)
    loglik = 0.0
    for j in range(Y.shape[1]):
        loglik += np.log(gp_marginal_likelihood(Y[:, j], x, l, s))  # independent observations
    return loglik

In [47]:
Y = np.zeros((100, 100))
x = np.linspace(0, 1, 100)

In [59]:
start = time.time()
for i in range(1):
    print(calculate_p(0.12, 1.0, Y, x, 0))
end = time.time()
print(end - start)

49366.43833403917
3.74096107483


In [57]:
def calculate_p_new(l, s, Y, x, prior_params):
    t, n = Y.shape
    cov = kernel_covariance(x, l, s)
    inverse = np.linalg.inv(cov)
    loglik = 0.0
    sign, logdet = np.linalg.slogdet(cov)
    constant = -0.5 * logdet - 0.5 * t * np.log(2 * np.pi)
    for j in range(Y.shape[1]):
        loglik += -0.5 * np.matmul(np.matmul(Y[:, j].reshape((1, t)), inverse), Y[:, j].reshape((t, 1)))[0][0] + constant
    return loglik

In [60]:
start = time.time()
for i in range(1):
    print(calculate_p_new(0.12, 1.0, Y, x, 0))
end = time.time()
print(end - start)

49366.438333831335
0.0406079292297
